In [1]:
import re
import numpy as np
import pandas as pd
import nltk
import pyforest
import texthero as hero
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#importing the data
data = pd.read_csv('../inputs/fake_job_postings.csv')
target = data['fraudulent']
data.drop(['job_id','salary_range','department','benefits'],axis=1,inplace=True)

In [3]:
#removing target values form dataset
data.drop(['fraudulent'],axis=1,inplace=True)

In [6]:
#filling null values 
data['required_education'].fillna('Unspecified',inplace = True)
data['employment_type'].fillna('no_info_about_employment',inplace = True)
data['required_experience'].fillna('experience_not_asked',inplace = True)
data['industry'].fillna('industry_not_given',inplace = True)
data['function'].fillna('function_not_given',inplace = True)

In [7]:
dataset = data.copy()

In [8]:
#dealing catogorical data
cat_cols = ['employment_type','required_experience','required_education','industry','function']

In [9]:
encoded = pd.get_dummies(data.employment_type)

In [10]:
encoded

,Contract,Full-time,Other,Part-time,Temporary,no_info_about_employment
0,0,0,1,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,0,1
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
17875,0,1,0,0,0,0
17876,0,1,0,0,0,0
17877,0,1,0,0,0,0
17878,1,0,0,0,0,0


In [11]:
encoded = pd.get_dummies(data.required_experience)
encoded

,Associate,Director,Entry level,Executive,Internship,Mid-Senior level,Not Applicable,experience_not_asked
0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
17875,0,0,0,0,0,1,0,0
17876,0,0,0,0,0,1,0,0
17877,0,0,0,0,0,0,0,1
17878,0,0,0,0,0,0,1,0


In [12]:
encoded = pd.get_dummies(data.required_education)
encoded

,Associate Degree,Bachelor's Degree,Certification,Doctorate,High School or equivalent,Master's Degree,Professional,Some College Coursework Completed,Some High School Coursework,Unspecified,Vocational,Vocational - Degree,Vocational - HS Diploma
0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,0,0,0,0,0,0,0,0,0,1,0,0,0
17876,0,1,0,0,0,0,0,0,0,0,0,0,0
17877,0,0,0,0,0,0,0,0,0,1,0,0,0
17878,0,0,0,0,0,0,1,0,0,0,0,0,0


In [40]:
cols = pd.get_dummies(data.industry).columns

In [41]:
cols

Index(['Accounting', 'Airlines/Aviation', 'Alternative Dispute Resolution',
       'Animation', 'Apparel & Fashion', 'Architecture & Planning',
       'Automotive', 'Aviation & Aerospace', 'Banking', 'Biotechnology',
       ...
       'Transportation/Trucking/Railroad', 'Utilities',
       'Venture Capital & Private Equity', 'Veterinary', 'Warehousing',
       'Wholesale', 'Wine and Spirits', 'Wireless', 'Writing and Editing',
       'industry_not_given'],
      dtype='object', length=132)

In [42]:
f = open("industry.txt", "a")
line_1 = "if param == " + "\""+cols[0]+"\":"
line_2 = "arr[" + str(0)+"] = 1"
line_3 = "return arr"
f.write(line_1+"\n\t"+line_2+"\n\t"+line_3+"\n")
for i in range(1,len(cols)):
    line_1 = "elif param == " + "\""+cols[i]+"\":"
    line_2 = "arr[" + str(i)+"] = 1"
    line_3 = "return arr"
    f.write(line_1+"\n\t"+line_2+"\n\t"+line_3+"\n")
f.close()

In [23]:
arr = [0 for i in range(len(cols))]

In [45]:
s = pd.Series(['jalkjfljkldjfl34jl3j3l90gdikl zklgfjlfkjg4  4 545'])

In [48]:
k.pipe(hero.clean)[0]

'jalkjfljkldjfl34jl3j3l90gdikl zklgfjlfkjg4'